<a href="https://colab.research.google.com/github/hyeonsang010716/Recommendation-system/blob/main/Pearson_correlation_%26_jaccard_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-25m.zip

--2024-03-13 04:41:18--  https://files.grouplens.org/datasets/movielens/ml-25m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261978986 (250M) [application/zip]
Saving to: ‘ml-25m.zip’

ml-25m.zip          100%[===================>] 249.84M   110MB/s    in 2.3s    

2024-03-13 04:41:21 (110 MB/s) - ‘ml-25m.zip’ saved [261978986/261978986]



In [ ]:
!unzip ml-25m.zip

Archive:  ml-25m.zip
   creating: ml-25m/
  inflating: ml-25m/tags.csv         
  inflating: ml-25m/links.csv        
  inflating: ml-25m/README.txt       
  inflating: ml-25m/ratings.csv      
  inflating: ml-25m/genome-tags.csv  
  inflating: ml-25m/genome-scores.csv  
  inflating: ml-25m/movies.csv       


In [ ]:
!head ml-25m/ratings.csv

userId,movieId,rating,timestamp
1,296,5.0,1147880044
1,306,3.5,1147868817
1,307,5.0,1147868828
1,665,5.0,1147878820
1,899,3.5,1147868510
1,1088,4.0,1147868495
1,1175,3.5,1147868826
1,1217,3.5,1147878326
1,1237,5.0,1147868839


In [ ]:
import csv

titles = dict()
genres = dict()
with open("ml-25m/movies.csv" , "r") as f:
  print(f.readline())
  csvreader = csv.reader(f)
  for mid , title , genre in csvreader:
    titles[int(mid)] = title
    genres[int(mid)] = set(genre.split("|"))


movieId,title,genres



In [ ]:
def jaccard_similarity(a,b):
  if len(a|b) == 0: return 0
  return len(a & b) / len(a | b)


def get_topk_jaccard_genres(target_mid , k = 20):
  target_genreset = genres[target_mid]
  res = []
  for mid , title in titles.items():
    genreset = genres[mid]
    score = jaccard_similarity(target_genreset , genreset)
    res.append((score , title))

  res.sort(reverse=True)
  return res[:k]



In [ ]:
mid = 164909
res = get_topk_jaccard_genres(mid , 20)

print(res)

[(1.0, 'Собака Павлова (2005)'), (1.0, 'Şabaniye (1984)'), (1.0, 'Zus & Zo (2001)'), (1.0, 'Zero (2018)'), (1.0, 'Zack and Miri Make a Porno (2008)'), (1.0, 'Youth in Revolt (2009)'), (1.0, 'You and the Night (2013)'), (1.0, 'You Stupid Man (2002)'), (1.0, 'You Are the Apple of My Eye (2011)'), (1.0, 'Yo Yo (Yoyo) (1965)'), (1.0, 'Yes, But... (Oui, mais...) (2001)'), (1.0, 'Yes Or No (2010)'), (1.0, 'Yellow Cab Man, The (1950)'), (1.0, 'Year of the Dog (2007)'), (1.0, 'Year by the Sea (2016)'), (1.0, "X's & O's (2007)"), (1.0, 'Wyjazd Integracyjny (2011)'), (1.0, 'World According to Garp, The (1982)'), (1.0, 'Working Girl (1988)'), (1.0, 'Words and Pictures (2013)')]


In [ ]:
ratings = []
with open("ml-25m/ratings.csv" , "r") as f:
  print(f.readline())
  for line in f:
    uid , mid , rating , timestamp = line.split(",")
    ratings.append((int(uid) , int(mid) , float(rating)))

ratings


In [ ]:
from collections import defaultdict
# movie를 키로 하고 user를 집합에 넣어준다.
usets = defaultdict(set)
#defaultdict는 비어있는 set으로 미리 만들어 준다.
for uid , mid, rating in ratings:
  usets[mid].add(uid)


In [ ]:
def get_topk_jaccard_ratings(target_mid , k = 20):
  target_uset = usets[target_mid]
  res = []
  for mid , title in titles.items():
    genreset = usets[mid]
    score = jaccard_similarity(target_uset , genreset)
    res.append((score , title))

  res.sort(reverse=True)
  return res[:k]

res = get_topk_jaccard_ratings(112552)
res

[(1.0, 'Whiplash (2014)'),
 (0.34346657709103123, 'Gone Girl (2014)'),
 (0.33069178628389156, 'The Imitation Game (2014)'),
 (0.3202530162820663, 'Interstellar (2014)'),
 (0.31890598662389213, 'Grand Budapest Hotel, The (2014)'),
 (0.3178914625340221, 'Ex Machina (2015)'),
 (0.31770758896670004,
  'Birdman: Or (The Unexpected Virtue of Ignorance) (2014)'),
 (0.3133357646486749, 'Her (2013)'),
 (0.312987900078064, 'Wolf of Wall Street, The (2013)'),
 (0.3059296340689558, 'Nightcrawler (2014)'),
 (0.2975845919065411, 'Mad Max: Fury Road (2015)'),
 (0.2957345586142398, 'Django Unchained (2012)'),
 (0.2737903411445145, 'The Martian (2015)'),
 (0.2668467981626587, 'Gravity (2013)'),
 (0.2627047459050819, 'Shutter Island (2010)'),
 (0.2575789399985997, 'Dallas Buyers Club (2013)'),
 (0.25711628659226915, 'Dark Knight Rises, The (2012)'),
 (0.25615628009354796, 'The Revenant (2015)'),
 (0.2561167699254437, 'Intouchables (2011)'),
 (0.25411334552102377, 'Arrival (2016)')]

In [ ]:
ursets = defaultdict(dict)
for uid , mid , rating in ratings:
  ursets[mid][uid] = rating

In [ ]:
for mid , urset in ursets.items():
  avg = sum(urset.values()) / len(urset)
  for k in urset:
    urset[k] -= avg

In [ ]:
def consine_similarity(a,b):
  nu = sum(a[k] * b[k] for k in a.keys() & b.keys())
  de = (sum(x * x for x in a.values()) * sum(x * x for x in a.values())) ** 0.5
  if de == 0 : return 0
  return nu / de

def get_topk_pearson_ratings(target_mid , k = 20):
  target_urset = ursets[target_mid]
  res = []
  for mid , title in titles.items():
    genreset = ursets[mid]
    score = consine_similarity(target_urset , genreset)
    res.append((score , title))

  res.sort(reverse=True)
  return res[:k]

res = get_topk_pearson_ratings(112552)
res

[(0.9999999999999988, 'Whiplash (2014)'),
 (0.19111707278803156, 'Inception (2010)'),
 (0.18189820107301488, 'Interstellar (2014)'),
 (0.17767282750285202, 'Dark Knight, The (2008)'),
 (0.1715264009485277, 'Django Unchained (2012)'),
 (0.16837186600682544, "Schindler's List (1993)"),
 (0.16465185561347476, 'Inglourious Basterds (2009)'),
 (0.1642028420675276, 'Fight Club (1999)'),
 (0.15668083853137563, 'Silence of the Lambs, The (1991)'),
 (0.1535912306698802, 'Pulp Fiction (1994)'),
 (0.15333147736969344, 'Gone Girl (2014)'),
 (0.15194389899169458,
  'Lord of the Rings: The Return of the King, The (2003)'),
 (0.15032309716277953, 'Matrix, The (1999)'),
 (0.15000561154629138, 'Shawshank Redemption, The (1994)'),
 (0.14820986101018202, 'Prestige, The (2006)'),
 (0.14312438143448764, 'Eternal Sunshine of the Spotless Mind (2004)'),
 (0.14302689166039786, 'Grand Budapest Hotel, The (2014)'),
 (0.14278151067272393, 'Wolf of Wall Street, The (2013)'),
 (0.1410801538193733, 'Her (2013)'),
 